# Intermediate Modeling: Return of the Widgets

In Opti101 we first learned about mathematical optimzation through a fictitious problem about producing and distributing widgets. If you would like to have the first notebook using this example from the previous training handy, you can find it [here](https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/optimization101/Modeling_Session_1/completed_modeling1.ipynb).

We are going to expand on this problem, so let's do a quick recap. We make widgets. They are produced in one of **five production facilities** and are then sent to one of **six distribution hubs** to be sold locally. Each distribution center has a demand forecast and each production facility has a min and max number of widgets it can make during this period.

Our production and distribution `sets` are:
- $P = \{\texttt{'Baltimore', 'Cleveland', 'Little Rock', 'Birmingham', 'Charleston'}\}$ $\quad\quad\quad\quad\quad\quad\quad\space\space \texttt{production}$
- $D = \{\texttt{'Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St. Louis'}\} \quad\quad\quad \texttt{distribution}$

Model `parameters`:
- $c_{p,d}$: cost to ship a widget from $p$ to $d$, $\quad\quad\quad\quad\quad\quad\quad\quad\texttt{transp_cost[p,d]}$
- $m_p$: maximum a production facility $p$ can produce, $\quad\quad\quad\quad\quad\quad\space\texttt{max_prod[p]}$
- $n_d$: demand at distribution hub $d$, $\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\texttt{n_demand[d]}$

If you are running this in Colab or don't have a Gurobi license, quickly installing `gurobipy` will install a limited license.

In [1]:
%pip install gurobipy

# Import packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 58.5 MB/s eta 0:00:00


In [2]:
# read in transportation cost data
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv')

# get production and distribution locations from data frame
production = list(transp_cost['production'].unique())
distribution = list(transp_cost['distribution'].unique())
transp_cost = transp_cost.set_index(['production','distribution']).squeeze()

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")
# the min prodcution is a fraction of the max
frac = 0.75

### The First Model

- Our **decision variables** are the amount produced at facility $p$ and shipped to distribution center $d$, denoted $x_{p,d}$
- We have **constraints** to ensure:
  - Min and max production
  - Demand is met
- The **objective** is to meet demand at **minimal cost**.

As a reminder, here is the original *formulation*:

\begin{align*}
{\rm min} &\sum_{p,d}c_{p,d}x_{p,d}\\
{\rm s.t.}\\
&\sum_{d}x_{p,d} \le m_p, &\forall p \in P \quad &\texttt{can}\_\texttt{produce[p]}\\
&\sum_{d}x_{p,d} \ge a*m_p,&\forall p \in P \quad &\texttt{must}\_\texttt{produce[p]}\\
&\sum_{p}x_{p,d} \ge n_d, &\forall d \in D \quad &\texttt{meet}\_\texttt{demand[d]}\\
&x_{p,d} \ge 0,  &\forall p \in P, d \in D\quad &\texttt{non-negativity}\\
\end{align*}

In [3]:
# gurobipy code for thie above formulation
m = gp.Model('widgets')

# decision vars
x = m.addVars(production, distribution, name = 'prod_ship')

# constraints
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
meet_demand = m.addConstrs(x.sum('*', d) >= n_demand[d] for d in distribution)

#objective
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

Restricted license - for non-production use only - expires 2024-10-28


Solve the optimization problem, and look at the results.

In [4]:
m.optimize()

x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
obj0 = m.getObjective()
obj0_value = obj0.getValue()

print(f"The original model had a total cost of {round(obj0_value,2)}")
sol[sol.shipment > 0]

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x6eb01f9f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.704890000e+03
The original model had a total cost of 1704.89


cost  shipment
production  distribution                
Baltimore   Nashville     5.96      19.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Nashville     4.13       2.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      80.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

### Fixing the very small shipment value
In the Intermediate Modeling recorded presentation, we saw a decision that may not be actually doable in an optimal solution.

$x_{Cleveland,Nashville} = 2$, meaning we have a shipment of only two units. It's *possible* that this is something we may need to avoid for various reasons, say for instance the transportation cost per unit is only valid for a minimum amount.




First, let's work through some of the logic in order to address this. The statement we want to model in words:

"If widgets are shipped from $p$ to $d$, then it must be at least $C$ units."

Let's replace some of the words with our decision variables and inequalities.

##### If _________, then _________.

If $x_{p,d} > 0$, then $x_{p,d} \ge C$.

##### Let's rewrite the original statement using the "IF-THEN to OR" equivalence mentioned in the previous session.


?????
$x_{p,d} \le 0$ or $x_{p,d} \ge C$.

Now let's replace the words again with our notation.
$$
x_{p,d} \le 0 \quad \texttt{or} \quad x_{p,d} \ge C
$$

We know that $x_{p,d}$ is already non-negative, so how can we strenghten what we have above?

##### Now, how do we actually model this?
We can do this in a few different ways:
- Big M
- Gurobi's indicator constraints
- Using a different variable type in Gurobi!

#### Using Big M Constraints
Let's first look at this in a general way, so we won't use our problem's subscripts initially. We could also take the more formulaic approach outlined in the previous session to build this, but there are times (with a little experience) where these can be built more intuitively. We will work through modeling
$$
x \le 0 \quad \texttt{OR} \quad x \ge C
$$

Since we have two clauses in the `OR` statement, let's use two [*auxiliary* variables](https://support.gurobi.com/hc/en-us/community/posts/10621090066321-What-is-exactly-auxiliary-variables-in-the-opt-model-), $z_1$ and $z_2$.

The inequality to the left of the `OR` is $x \le 0$. Remember the general point is to enforce a desired constraint when the binary variable takes a specific value, and then implement no restrictions when it takes the opposite value.

##### Clause 1 of `OR` statement:
$$
x \le M_1 \times z_1
$$

If $z_1 = 0$, then $x \le 0$. If $z_1 = 1$, then $x \le M_1$. Assuming $M_1$ is large enough, then there is no resrtiction on $x$.

For the right side of the `OR` statement, we have $x \ge C$. Again, in one case of the auxiliary variable we want the desired inqueality, and the other case to put no restrictions on our variable.

$$
x + M_2 \times z_2 \ge C
$$

If $z_2 = 0$, then $x \ge C$. If $z_2 = 1$, then (as long as $M_2 \ge C$) there is no restriction on $x$.

Next, we need to see how the auxiliary variables need to work together in this case. Since this is an `OR` statement, one of these needs to happen, so this gives us:
$$
z_1 + z_2 \ge 1
$$
And can both of these happen together (i.e. can $z_1 = z_2 = 1$)? Nope. So we can strengthen this to:
$$
\begin{align*}
z_1 + z_2 &= 1\\
z_1 &= 1-z_2
\end{align*}
$$
Let's substitute this into the above inequality and since we have only one auxiliary variable in this simplification we can just call it $z$.
$$
\begin{align*}
&x \le M_1 \times (1-z)\\
&x + M_2 \times z \ge C
\end{align*}
$$

OK - now back to our problem. We need to define the new *auxiliary* binary variable for each **production facility** and **distribution center**, so we will call it $z_{p,d}$. This gives us:
$$
\begin{align*}
&x_{p,d} \le M^1_{p,d} \times (1-z)\\
&x + M^2_{p,d} \times z \ge C
\end{align*}
$$

Lastly, before we get to actually implementing this. Let's figure out some good choices for the $M$s above. Looking at $x_{p,d} \le M^1_{p,d}$, can we think what else is an *upper-bound* on $x_{p,d}$?

A good choice would be the maximum amount that a production facility can produce, $m_p = \texttt{max_prod[p]}$ (Don't get $m$ and $M$ confused!). $C$ is a clear choice for each $M^2_{p,d}$. Alright, let's code it by adding the auxiliary variables first.

In [5]:
z = m.addVars(production, distribution, vtype=GRB.BINARY, name = 'min_distribution')

Next, let's add the constraints developed above.

In [6]:
min_ship1 = m.addConstrs(x[p,d] <= max_prod[p]*(1-z[p,d]) for p in production for d in distribution)
m.update()

Ah, we still need to decide on a value for $C$. We'll set it to 10 for now.

In [7]:
C = 30
min_ship2 = m.addConstrs(x[p,d] + C*z[p,d] >= C for p in production for d in distribution)
m.update()

Solve and look at the new solution.

In [8]:
m.optimize()

x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
obj1 = m.getObjective()
obj1_value = obj1.getValue()
sol[sol.shipment > 0]

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 76 rows, 60 columns and 210 nonzeros
Variable types: 30 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 2e+02]
Found heuristic solution: objective 2825.4400000
Presolve time: 0.00s
Presolved: 76 rows, 60 columns, 210 nonzeros
Variable types: 30 continuous, 30 integer (30 binary)

Root relaxation: objective 1.704890e+03, 34 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1704.89000    0    2 2825.44000 1704.89000  39.7%     -    0s
H    0     0                    1748.4200000 170

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      30.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      71.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

In [9]:
print(f"The updated model had a total cost of {round(obj1_value,2)}.")
print(f"The additional cost is {round(obj1_value - obj0_value,2)}.")

The updated model had a total cost of 1748.42.
The additional cost is 43.53.


#### Using Gurobi's Indicator Constraint
That was A LOT of work, you must be thinking "Since this is formulaic, is there a better way?" Well the answer is **yes**. This is where [indicator constriaints](https://www.gurobi.com/documentation/current/refman/py_model_agc_indicator.html) can be very useful.

These allow us to be more direct in modeling `IF-THEN` statements. These constraints work by utilizing a single `binary` decision variable as the **IF**, with another constraint as the **THEN**.

This boils down to "If the binary variable equals 1, then enforce the constraint. If it's 0, then don't."

Because of this, we still need to define *auxiliary* variables $z$, but from there it's a bit simpler.

In [10]:
# remove the big M constraints and variable
m.remove([min_ship1, min_ship2])
C = 30
# overloaded form
zis1 = m.addConstrs((((z[p,d] == 1) >> (x[p,d] >= C)) for p in production for d in distribution), name = "zis1")
zis0 = m.addConstrs((((z[p,d] == 0) >> (x[p,d] <= 0)) for p in production for d in distribution), name = "zis0")
m.update()

In [11]:
m.optimize()

x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
sol[sol.shipment > 0]

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 60 columns and 90 nonzeros
Model fingerprint: 0x6accf263
Model has 60 general constraints
Variable types: 30 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+01, 2e+02]
  GenCon rhs range [3e+01, 3e+01]
  GenCon coe range [1e+00, 1e+00]

MIP start from previous solve did not produce a new incumbent solution

Presolve added 60 rows and 0 columns
Presolve time: 0.00s
Presolved: 76 rows, 60 columns, 210 nonzeros
Variable types: 30 continuous, 30 integer (30 binary)
Found heuristic solution: objective 2998.0600000

Root relaxation: objective 1.704890e+03, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      30.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      71.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

This solution looks awfully familiar...and was pretty easy to model! Can it get easier?

#### Using Semi-Continuous Variables
This type of constraint, where we want a decision variable $x$ such that
$$
x=0 \quad \texttt{or} \quad l \le x \le u
$$
we can define $x_{p,d}$ as a `semi-continuous` decision variable and specify the corresponding *lower* and *upper* bounds

In [12]:
m.remove([zis1, zis0])

Let's rewrite the whole model using the `semi-continuous` variable and solve one more time.

In [13]:
# decision vars
m = gp.Model('widgets')

# decision vars
x = m.addVars(production, distribution, vtype=GRB.SEMICONT, lb = C, name = 'prod_ship')

# constraints
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
meet_demand = m.addConstrs(x.sum('*', d) >= n_demand[d] for d in distribution)

# objective
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)
m.optimize()

# extract and show solution
x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
sol[sol.shipment > 0]

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x27f83804
Variable types: 0 continuous, 0 integer (0 binary)
Semi-Variable types: 30 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [3e+01, 3e+01]
  RHS range        [6e+01, 2e+02]
Presolve time: 0.00s
Presolved: 76 rows, 60 columns, 210 nonzeros
Variable types: 30 continuous, 30 integer (30 binary)

Root relaxation: objective 1.704890e+03, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1704.89000    0    2          - 1704.89000      -     -    0s
H    0     0             

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      30.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      71.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

## Constraining production facilities

For this part of the notebook, we will use different numbers for the maximum amount a facility can produce.

In [14]:
max_prod2 = pd.Series([210,225,140,130,220], index = production, name = "max_production")

We also have to make one more substantial change to the model now that all facilities are not required to be open. The $\space\texttt{must}\_\texttt{produce[p]}$ contraint needs to be removed since this set of constraints, as written, will make all production facilities open. Let's define a new model, call it `m2`, and code up the parts we want to port over from the first model.

In [15]:
m2 = gp.Model('widgets2')

# decision vars
x = m2.addVars(production, distribution, name = 'prod_ship')

# constraints
can_produce = m2.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod2[p] for p in production), name = 'can_produce')
meet_demand = m2.addConstrs(x.sum('*', d) >= n_demand[d] for d in distribution)

# define total cost for quick access later
total_cost = gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution)

# objective
m2.setObjective(total_cost, GRB.MINIMIZE)


Let's run the optimization to establish a baseline before using the new decision variables.

In [16]:
m2.optimize()
x_values = pd.Series(m2.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x_values], axis=1)
sol2[sol2.shipment > 0]

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 11 rows, 30 columns and 60 nonzeros
Model fingerprint: 0xa467002a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 2e+02]
Presolve time: 0.01s
Presolved: 11 rows, 30 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.030000e+02   0.000000e+00      0s
       8    1.5984100e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.598410000e+03


cost  shipment
production  distribution                
Baltimore   Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53     101.0
            St. Louis     4.01      29.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      12.0

Now add the new decision variables to the model.

In [17]:
y = m2.addVars(production, vtype=GRB.BINARY, name = 'prod_on')

As discussed in the previous session, we need to be a little careful when linking $x_{p,d}$ and $y_p$ decision varibles. Let's use the `Big-M` approach. Using indicator constraints is part of the exercises.

In [18]:
m2.addConstrs((x[p,d] <= max_prod2[p]*y[p] for p in production for d in distribution), name = 'xy_link')
m2.update()

 ### Using binary decision variables to open production facilities

 While we have linked distribution to production, we haven't implemented anything that restricts the `m2` optimal solution, so let's do that. Here are the three ideas specific to this problem that we'll dive into:
 - Regionality restrictions
 - Minimal number of facilities and strict limits
 - Maximize the minimum shipment

### Regional restrictions
One can come up with a number of reasons why a company making widgets, or anything, would like to implement some restrictions on where items are produced and distributed. For example, our model doesn't consider things like workforce in an area or getting raw materials. Each of these can have big impacts on decision making.

The leadership at *Brigitte's Widgets* (yes, I finally named this company...I feel like they earned it) wrote in an email that we are to make sure that if the production facility in *Charleston* is open then *Cleveland* cannot be open and *Baltimore* cannot be open.

Using $y_{city \space name}$ , we can write this statement as:
$$
\texttt{If}\space y_{Charelston} = 1 \texttt{, then}\space y_{Cleveland}=0  \space \texttt{and}\space y_{Baltimore} = 0
$$

Given what we have learned about how to use indicator constraints, this seems like the best approach. What goes to the left of the `>>` symbol is straightforward, but what should the right side look like?

##### What constraint is needed?
$$
\texttt{If}\space
y_{Charleston} = 1\space \texttt{, then}\space y_{Cleveland} + y_{Baltimore} = 0
$$

In [19]:
# regionality conditions
reg_cond = m2.addConstr((y['Charleston']==1) >> (y['Cleveland'] + y['Baltimore'] == 0))
m2.write('reg_cond.lp')

It's always a good idea to write the `.lp` file. It's a great way to see if your model has any obvious errors.

Now let's solve and look at the solution.

In [20]:
m2.optimize()
x_values = pd.Series(m2.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x_values], axis=1)
sol2[sol2.shipment > 0]

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 41 rows, 35 columns and 120 nonzeros
Model fingerprint: 0x53227d34
Model has 1 general constraint
Variable types: 30 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 12 rows and 4 columns
Presolve time: 0.00s
Presolved: 29 rows, 31 columns, 99 nonzeros
Variable types: 30 continuous, 1 integer (1 binary)
Found heuristic solution: objective 1878.9800000

Root relaxation: cutoff, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

cost  shipment
production  distribution                
Baltimore   Lexington     4.33      92.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Lexington     2.54      29.0
            St. Louis     4.88      12.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53     101.0
            St. Louis     4.01      29.0

In [21]:
# remove the regional condition constraint from the model
m2.remove(reg_cond)

### Minimal number of facilities
Given we have our link between distribution variables $x$ and production variables $p$ it's quite simple to see the minimal number of facilities needed to meet demand by setting the objective to minimize the sum over $y$.

$$
{\rm min} \sum_p y_p
$$

In [22]:
m2.setObjective(y.sum())
m2.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 41 rows, 35 columns and 120 nonzeros
Model fingerprint: 0x31d55d44
Variable types: 30 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]

MIP start from previous solve produced solution with objective 4 (0.01s)
Loaded MIP start from previous solve with objective 4

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 39 rows, 35 columns, 121 nonzeros
Variable types: 30 continuous, 5 integer (5 binary)

Root relaxation: cutoff, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

In order to meet demand, we need at least four production facilities. Let's compare the total cost to the baseline for this scenario.

In [23]:
# (new objective value) - (previous objecive value)
total_cost.getValue() - sum(sol2.cost*sol2.shipment)

152.55999999999995

Does this really say anything (for certain) about the cost of using only four facilities?

No, since we didn't specify any way to prioritize between them there could be more than one set of four that can meet demand with different costs. Let's set the number of **production facilities to be at most four** and then minumize costs like we were doing before.

In [24]:
only_four = m2.addConstr(y.sum() <= 4)
m2.setObjective(total_cost, GRB.MINIMIZE)
m2.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 42 rows, 35 columns and 125 nonzeros
Model fingerprint: 0xd0a037fc
Variable types: 30 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+02]

MIP start from previous solve produced solution with objective 1878.98 (0.01s)
Loaded MIP start from previous solve with objective 1878.98

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 40 rows, 35 columns, 126 nonzeros
Variable types: 30 continuous, 5 integer (5 binary)

Root relaxation: objective 1.662450e+03, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

Now let's compare again the total costs.

In [25]:
round(total_cost.getValue() - sum(sol2.cost*sol2.shipment),2)

-216.53

The cost only increases by a relatively small amount even though we are only using four facilities. The lesson here is only infer what your objective and constraints allow.

This seems like we stumbled upon a handy way to **handle multiple objectives**. I wonder if we'll talk more about this in later sessions.

In [26]:
#remove the contraing limiting to four facilities
m2.remove(only_four)

### Maximize the minimum number of widgets shipped
There could be a priority to make sure *Brigitte's Widgets* transports as much as possible per shipment. One way to interpret this is to maximize the minimum shipment, which means to increase the smallest $x_{p,d}$ as much as possible.

Let's set $r = {\rm min}_{p\in P, d\in D}\{x_{p,d}\}$ and then define the objective to be
${\rm max}\space r$

Let's use general constraints to model this.

In [27]:
# overloaded forms
r = m2.addVar(vtype=GRB.INTEGER, name = 'r')
#minconstr = m2.addConstr(r == min_([x[p,d] for p in production for d in distribution]), name="minconstr")
minconstr = m2.addGenConstrMin(r, [x[p,d] for p in production for d in distribution],name= "minconstr")
m2.setObjective(r, GRB.MAXIMIZE)
m2.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 41 rows, 36 columns and 120 nonzeros
Model fingerprint: 0x26f08371
Model has 1 general constraint
Variable types: 30 continuous, 6 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]

MIP start from previous solve produced solution with objective -0 (0.01s)
Loaded MIP start from previous solve with objective -0

Presolve added 1 rows and 25 columns
Presolve time: 0.00s
Presolved: 42 rows, 61 columns, 161 nonzeros
Variable types: 30 continuous, 31 integer (30 binary)

Root relaxation: objective 2.100000e+01, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  De

In [28]:
m2.remove(minconstr)

In this case, we don't need the `min` general constraint. We can get away with
$$
r\le x_{p,d}, \quad ∀p \in P, d \in D.
$$

In [29]:
m2.addConstrs(r <= x[p,d] for p in production for d in distribution)
m2.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 71 rows, 36 columns and 180 nonzeros
Model fingerprint: 0xbf3bad03
Variable types: 30 continuous, 6 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]

MIP start from previous solve produced solution with objective 21 (0.01s)
Loaded MIP start from previous solve with objective 21

Presolve removed 30 rows and 5 columns
Presolve time: 0.00s
Presolved: 41 rows, 31 columns, 120 nonzeros
Variable types: 30 continuous, 1 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 21 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.100000000

Since we are maximizing $r$, we don't need to worry about specifying what bounds $r$ from below and the contraints we added are enough to model this relationship in this case.

### On to the next session!
It's time for a short break and for you to put your knowlege to the test!